## Naive implementation of a production circuit

In the naive implementation of a circuit, we have a population of cells ($X_P$) that are producing the protein/product of interest with some rate constant $\beta$, and as a result have some burden/growth penalty that is reflected in a growth rate $\mu_P < \mu_N$. Here $\mu_P$ is the growth rate of producers, and $\mu_N$ is the growth rate of non-producer cells. Producer cells can mutate with some probability $\lambda_{m_B}$ (mutation that removes burden) to non-producer cells ($X_N$) that have a growth rate $\mu_N$. Looking at this in the deterministic case in continuous culture with a carrying capacity K, and assuming the rate of production is limited in the same manner as the growth rate of the cells, we have that:

\begin{align}
\frac{dX_P}{dt} = \mu_P X_P \frac{K - X_P - X_N}{K} - \lambda_{MB} X_P - DX_P
\end{align}

\begin{align}
\frac{dX_N}{dt} = \mu_N X_N \frac{K - X_P - X_N}{K} + \lambda_{MB} X_P - DX_N
\end{align}

\begin{align}
\frac{dP}{dt} = \beta_P X_P \frac{K - X_P - X_N}{K}
\end{align}

$X_P$: Number of producer cells

$X_N$: Number of non-producer cells

$P$: Total production

$\lambda_{m_B}$: rate of mutation that removes burden (1/h)

$\beta_P$: Production rate (1/h)

## Implementation of differentiation

With differentiation architecture, there are now two possible mutation types which could occur:


- Production mutation ($\lambda_{MB}$): Mutation which inactivates the production capability of the cell. When a progenitor cell with this mutation differentiates, it will be a non-producer and growth at rate $\mu_N$.


- Differentation mutation ($\lambda_{MD}$): Mutation which inactivates the progenitor cell ability to differentiate. It will then grow at rate $\mu_N$ and differentiate at rate 0.

With the implementation of differentiation alone where differentiated cells many continue to divide indefinitely, differentiated cells may incur this first type of mutation ($\lambda_{M_B}$) to yield cells that grow with rate $\mu_N$, and whose differentiation does not have any effect. In writing out the system of ODEs to describe this, the subscript $PP$ will be used to denote progenitor cells that are capable of differentiating into productive cells, $N$ for non-producers, and $DP$ for differentiated producers. As the differentiation architecture itself requires very minimal protein expression, we make the simplifying assumption that all non-producer cells grow at the same WT growthrate.

\begin{align}
\frac{dX_{PP}}{dt} = \mu_N X_{PP} \frac{K - X_{PP} - X_P - X_N}{K} - \lambda_{diff}X_{PP} - (\lambda_{M_B} + \lambda_{M_D}) X_{PP} - DX_{PP}
\end{align}

\begin{align}
\frac{dX_{DP}}{dt} = \mu_P X_{DP} \frac{K - X_{PP} - X_P - X_N}{K} + \lambda_{diff}X_{PP} - \lambda_{M_B} X_{DP} - DX_{DP}
\end{align}

\begin{align}
\frac{dX_{N}}{dt} = \mu_N X_N \frac{K - X_{PP} - X_P - X_N}{K} + (\lambda_{M_B} + \lambda_{M_D}) X_{PP} + \lambda_{M_B} X_{DP} - DX_N
\end{align}

\begin{align}
\frac{dP}{dt} = \beta_P X_P \frac{K - X_{PP} - X_P - X_N}{K}
\end{align}

Where:

$X_{PP}$: Progenitor cells that will differentiate into productive differentiated cells.

$X_{DP}$: Differentiated producer cells.

$X_N$: Non-productive cells

$\lambda_{m_D}$: rate of mutation that destroys ability of progenitor cells to differentiate (1/h)

## Implementation of terminal differentiation

With the addition of selection against differentiated cells through selection for the R6K plasmid, the number of cell divisions is limited. Though differentiated cells retain limited proliferative potential, they will ultimately cease dividing and die, and we will therefor refer to this as 'terminal differentiation'. With this circuit architecture, we now consider three types of mutations that may occur:

1. Production mutation ($\lambda_{M_B}$): Mutation which inactivates the production capability of the cell. When a stem cell with this mutation differentiates, it will be a non-producer and growth at rate $\mu_N$.

2. Differentation mutation ($\lambda_{M_D}$): Mutation which inactivates the stem cells ability to differentiate. It will then grow at rate $\mu_N$ and be incapable of differentiation.

3. Selection mutation ($\lambda_{M_S}$): Mutation which occurs in differentiated cells (either producers or non-producers) which allows cells to evade selection and proliferate indefinitely.


In writing out the ODEs for this case, $P$ will denote a progenitor cell capable of differentiating, $D$ will denote a differentiated cell, and $M$ will denote a mutated progenitor cell that can no longer undergo differentiation. The subscript $N$ will indicate that a stem cell will differentiate into non-productive differentiated cells, or is currently a non-productive differentiated cell, and similarly $P$ for productive. Finally, the differentiated cells are not all equal, but are distinct in the number of divisions they have remaining before diluting the plasmid through cell division to the point where antibiotic resistance is insufficient to allow further proliferation. The cell produced from the differentiation event (i=0) will have 'n' divisions remaining, and when this cell divides, there will be one fewer cell with 'n' divisions remaining, and 2 more cells with'n-1' divisions remaining (i=1). When i=n, cells will no longer divide, and this is equivalent to cell death. Though the behavior of cells that are no longer able to divide (whether or not they continue to produce the protein/product of interest) would likely depend both on the type of selectable marker encoded on the R6K plasmid and the protein/product that is being produced, using chloramphenicol resistance as the selectable marker likely would prevent protein production in cells that are no longer able to divide.

As with the differentiation architecture, we make the assumption that all non-producer cells growth with the same WT growth rate. In the modeling of this architecture, we also must decide how selection will affect the growth rate of the differentiated cells. Though we could imagine cell growth slowing due to insufficient chlorR expressions as the plasmid copy number is reduced through cell division, in modeling this architecture we make the assumption that selection only affects cells with no remaining cell divisions. Therefor the growth rate of differentiated cells capable of further cell division is determined solely by the expression burden. With these assumptions, we can now write out our system of ODEs.



#### Progenitor cells:
\begin{align}
\frac{dX_{PP}}{dt} = \mu_{PP} X_{PP} \frac{K - X_{TOT}}{K} - \lambda_{diff}X_{PP} - \lambda_{MB}X_{PP} - \lambda_{MD} X_{PP} - DX_{PP}
\end{align}

\begin{align}
\frac{dX_{PN}}{dt} = \mu_{PN} X_{PN} \frac{K - X_{TOT}}{K} - \lambda_{diff}X_{PN} + \lambda_{MB}X_{PP} - \lambda_{MD} X_{PN} - DX_{PN}
\end{align}

#### Mutant cells:
\begin{align}
\frac{dX_{MP}}{dt} = \mu_{MP} X_{MP} \frac{K - X_{TOT}}{K} + \lambda_{MS} \sum_{i=0}^{i=n-1}{X_{DP}} - \lambda_{MB}X_{MP} -  DX_{MP}
\end{align}

\begin{align}
\frac{dX_{MN}}{dt} = \mu_{MN} X_{MN} \frac{K - X_{TOT}}{K} + \lambda_{MD} \left(X_{PN} + X_{PP}\right) + \lambda_{MB}X_{MP} + \lambda_{MS}\sum_{i=0}^{i=n-1}{X_{DN}} -  DX_{MN}
\end{align}

#### Productive differentiated cells: 

For $i=0$
\begin{align}
\frac{dX_{DP}(0)}{dt} = \lambda_{diff}X_{PP} - \mu_{DP}(0) X_{DP}(0)\frac{K - X_{TOT}}{K} - \lambda_{MB}X_{DP}(0) - \lambda_{MS}X_{DP}(0)- DX_{DP}(0)
\end{align}

For $0<i<n$
\begin{align}
\frac{dX_{DP}(i)}{dt} = 2\mu_{DP}(i-1) X_{DP}(i-1)\frac{K - X_{TOT}}{K} - \mu_{DP}(i) X_{DP}(i)\frac{K - X_{TOT}}{K} - \lambda_{MB}X_{DP}(i) - \lambda_{MS}X_{DP}(i) - DX_{DP}(i)
\end{align}

For $i=n$
\begin{align}
\frac{dX_{DP}(n)}{dt} = 2\mu_{DP}(n-1) X_{DP}(n-1)\frac{K - X_{TOT}}{K} - DX_{DP}(n)
\end{align}

#### Non-productive differentiated cells: 

For $i=0$
\begin{align}
\frac{dX_{DN}(0)}{dt} = \lambda_{diff}X_{PN} - \mu_{DN}(0) X_{DN}(0) + \lambda_{MB}X_{DP}(0) - \lambda_{MS}X_{DN}(0)- DX_{DN}(0)
\end{align}

For $0<i<n$
\begin{align}
\frac{dX_{DN}(i)}{dt} = 2\mu_{DN}(i-1) X_{DN}(i-1) - \mu_{DN}(i) X_{DN}(i) + \lambda_{MB}X_{DP}(i) - \lambda_{MS}X_{DN}(i) - DX_{DP}(i)
\end{align}

For $i=n$
\begin{align}
\frac{dX_{DN}(n)}{dt} = 2\mu_{DN}(n-1) X_{DN}(n-1) - DX_{DP}(n)
\end{align}

#### Production
Finally, we can model the production, assuming that all productive differentiated cells are producing at the same maximum rate. While we could model the concentration, we are mainly interested in the total output, and therefor will only keep track of the total production

\begin{align}
\frac{dP}{dt} = \beta_P \left(\sum_{i=0}^{i=n}X_{DP}(i) + X_{MP}\right)\frac{K - X_{TOT}}{K}
\end{align}

In all equations above, the total population ($X_{TOT}$) is given by:

$X_{TOT} =  X_{PP} + X_{PN} + X_{MP} + X_{MN} + \sum_0^n X_{DP}(i) + \sum_0^n X_{DN}(i)$

There may be two adjustments to this production equation, one of which will affect the effective total population size. Particularly in the case of chloramphenicol which acts by inhibiting translation, the differentiated cells of generation 'n' would not be metabolically active, would be considered dead, and therefor would not contribute either to the total effective population size or to the production of P. We would then have:

\begin{align}
\frac{dP}{dt} = \beta_P \left(\sum_{i=0}^{i=n-1}X_{DP}(i) + X_{MP}\right)\frac{K - X_{TOT}}{K}
\end{align}

The total effective population size ($X_{TOT}$) would then be given by:

$X_{TOT} =  X_{PP} + X_{PN} + X_{MP} + X_{MN} + \sum_0^{n-1} X_{DP}(i) + \sum_0^{n-1} X_{DN}(i)$

The second would come from relaxing the assumption that all producer cells would produced P at the same rate. We could imagine instead that, particularly in the case of chloramphenicol, the production rate of P would vary with the growth rate as this to is affected by the inhibition of translation by the antibiotic. We then would have:

\begin{align}
\frac{dP}{dt} = \beta_P \sum_{i=0}^{i=n-1}X_{DP}(i)\frac{\mu_{DP}(i)}{\mu_{{DP}_{max}}}\frac{K - X_{TOT}}{K} + \beta_P X_{MP}\frac{\mu_{DP}(i)}{\mu_{{DP}_{max}}}\frac{K - X_{TOT}}{K}
\end{align}

In the modeling done for this paper, we assume that all differentiated cells that are capable of additional cell divisions $(i<n)$ grow and produce with the same rate. We also assume differentiated cells that can no longer divide are equivalent to dead cells, and neither produce or contribute to the total cell count.

